In [3]:

import os
import sys
#add site-packages to the path

#sys.path.append("/home/marin/.local/lib/python3.10/site-packages")



import csv 
import matplotlib.pyplot as plt

import pandas as pd
import tqdm

import numpy as np
import plotly.graph_objects as go
parent_dir = os.path.join(os.getcwd())
print(parent_dir)



/mnt/c/Users/marin/Documents/Ma2/2024_C_AV_RPI-1/PythonTest


In [4]:




# There are multiple folders in the parent directory. We need to get csv file each folder 
# there is an unknown number of levels till we reach the csv files.
# do a recursive search for the csv files
csvList = list()
def get_files(extension ):

    for root, dirs, files in os.walk(parent_dir):
        for file in files:
            
            if file.endswith(extension):
                csvList.append(os.path.join(root, file))
    return csvList
    
    
    


def get_first_row(path):
    #print(f"Reading the first row of the file: {path}")
    with open(path, 'r') as file:
        data = file.read()
    # Split the data into lines
    lines = data.split('\n')
    var = lines[0].split(',')

    if len(var) < 2:
        var = lines[0].split(';')
    return var


def from_text_to_csv(path):
    new_path = path.replace('.txt', '.csv')
    last_var = path.split('/')[-1].replace('.txt', '')
    with open(new_path, 'w') as file:
        writer = csv.writer(file)
        writer.writerow(['Date', 'Time', last_var])
    
    # Read the text file
    with open(path, 'r') as file:
        data = file.read()
    # Split the data into lines
    lines = data.split('\n')
   
    # Create a progress bar
    progress_bar = tqdm.tqdm(total=len(lines), desc='Converting lines')
    
    for line in lines:
        elements = line.split(':')
        last_elem = elements[-1]
        tempLine = line.replace(last_elem, '')
       
        try:
            date, time = tempLine.split(' ')
        except ValueError:
            # Handle the error here
            print("Error occurred while splitting the line:", line)
            continue
        date,time = tempLine.split(' ')
        with open(new_path, 'a') as file:
            writer = csv.writer(file)
            writer.writerow([date, time, last_elem])

        
        # Update the progress bar
        progress_bar.update(1)
    

    # Close the progress bar
    progress_bar.close()

def var_Mapping():
    text_files = get_files('.csv')
    varMap = dict()
    for file in text_files:
        #add the file to the dictionary i
        tempVar = get_first_row(file)
        for i in tempVar:
            if i not in varMap:
                varMap[i] = [file]
            else:
                varMap[i].append(file)
    #for i in varMap:
    # make the variables green in color in the text
        #print(f"\033[92m{i}\033[0m :{varMap[i]} ")
    return varMap





def calculate_norm(acceleration):
    # Parse acceleration values from the CSV file
    accel_values = acceleration.strip('(').strip(')').split(',')
    accel_values = [value.strip().strip('(') for value in accel_values]

    accel_values = [float(value) for value in accel_values]
    
    # Calculate the norm of the acceleration vector
    norm = np.linalg.norm(accel_values)
    return norm






def plot_var(variables):
    varMap = var_Mapping()
    # Get the acceleration data from the CSV file
    acceleration_file = varMap[variables]
    # Read the CSV file
    data = pd.read_csv(acceleration_file)
    # Calculate the norm of the acceleration vector
    data['norm'] = data[variables].apply(calculate_norm)
    # Plot the norm of the acceleration vector with row position on the x-axis
    plt.plot(data['norm'])
    plt.xlabel('Row position')
    plt.ylabel(f'{variables}')
    plt.title(f'{variables} vector over time')

    #plt.show()
    #save in folder -> "plots" with the name of the file being the name of the variables
    if not os.path.exists('plots'):
        os.makedirs('plots')
    print(f"Saving plot as {variables}.png")
    plt.savefig(f'plots/{variables}.png')
    
    # Create a new plot
    fig = go.Figure()
    # Add a line to the plot
    fig.add_trace(go.Scatter(x=data.index, y=data['norm'], mode='lines', name=variables))
    # Update the layout of the plot
    fig.update_layout(title=f'{variables} vector over time', xaxis_title='Row position', yaxis_title=f'{variables} vector')
    # Save the plot as an HTML file
    fig.write_html(f'plots/{variables}.html')
    # Show the plot

#19289
    
def find_row_value(variables, row):
    varMap = var_Mapping()
    # Get the acceleration data from the CSV file
    acceleration_file = varMap[variables]
    # Read the CSV file
    data = pd.read_csv(acceleration_file)
    # Calculate the norm of the acceleration vector
    
    return data['Time(ms)'][row]





#plot_var('acceleration')

def reevaluate_timeStamp(std_time, paths):

    print (f"Standard time: {std_time}")
    print(f"Paths: {paths}")


    for path in paths:
        print(f"Reading the file: {path}")
        #read the csv file and change Time(s) to Time(ms) with panda
        data = pd.read_csv(path)
        #get the time column
        time = data['Time(ms)']
        #convert the time to ms
        time_ms = time.apply(lambda x: from_shitStamp_to_ms(x))
        #subtract the standard time from the time_ms

        new_time = time_ms - std_time
        #add the new time to the data
        data['Time(ms)'] = new_time
      
        #save the new data to a new file
        new_path = path.replace('.csv', '_new.csv')
        data.to_csv(new_path, index=False)
        print(f"New file created: {new_path}")

        


def from_shitStamp_to_ms(shitStamp):
    # Split the timestamp into its components
    components = shitStamp.split(':')
    # Remove the last empty string
    components.pop()
 
    # Convert the components to integers
    components = [float(component) for component in components]
    # Calculate the time in milliseconds
    try:
        time_ms = components[0] * 3600000 + components[1] * 60000 + components[2] * 1000
        return int(time_ms)
    except IndexError:
        print(f"Error occurred while converting the timestamp: {shitStamp},{components}")



def reavgnss_timeStamp(std_time, path):

    #std_time is in s

    std_time = std_time * 1000

    #multiply by 1000 to get the time in ms all elements in column Time(s) using panda 

    #read the csv file
    data = pd.read_csv(path)

    #get the time column
    print(f"Columns: {data.columns}")
    time = data['Time(s)']

    #convert the time to ms
    time_ms = time * 1000

    #subtract the standard time from the time_ms
    new_time = time_ms - std_time

    #add the new time to the data
    data['Time(ms)'] = new_time
    #remove the old time column
    data = data.drop(columns=['Time(s)'])





    #save the new data to a new file
    new_path = path.replace('.csv', '_new.csv')
    data.to_csv(new_path, index=False)
    print(f"New file created: {new_path}")

def mean_cycle(path):
    #read the csv file
    data = pd.read_csv(path)
   
    if 'Time(ms)' not in data.columns:
        print(f"Time(ms) not in {path}")
        return -1
    #get the time column
    time = data['Time(ms)']

    #between each n and n+1, calculate the difference and add it to a list
    time_0 = np.array(time)
    time_1 = np.roll(time, 1)

    #remove the first element of time_1 and the first element of time_0
    time_1 = time_1[1:]
    time_0 = time_0[1:]

    time_diff = time_0 - time_1
    print (f"Time diff: {time_diff[:10]}")
    #calculate the mean of the list
    mean_time_diff = np.mean(time_diff)
    

    return mean_time_diff




    
    

#from_shitStamp_to_ms('12:02:47.993748:')


#43367993
shitPaths = "./barometer.csv".split(';')

#reevaluate_timeStamp(43367993, shitPaths)

#reavgnss_timeStamp(2379.4135,"/mnt/c/Users/marin/Documents/Ma2/2024_C_AV_RPI-1/PythonTest/ERT_Wildhorn_Flight_Data/Team12_PFR_Altitude_SRAD Avionics_Logging_Data_EuRoC2022/avionics_data/gnss_data.csv")

cycleMap = dict()
print(f"\033[94m{list(map(lambda x: x.split('/')[-1],var_Mapping()['Time(ms)']))}\033[0m")
#print (f"{var_Mapping()['Time(ms)']}")
#for i in var_Mapping()['Time(ms)']:
    #cycleMap[i] = mean_cycle(i)

#print(f"\033[94m{cycleMap}\033[0m")

def add_to_dic(key ,dic, value):
    if key not in dic.keys():
        dic[key] = [value]
    else:
        dic[key].append(value)
    



    progress_bar.close()
    dataframeList = list_of_dataframes
    
    print(f"Combined data: {combined_data.columns}")
    #save to a csv file
    combined_data.to_csv('combined_data.csv', index=False)
    




['gnss_data_new.csv', 'accelerometer_new.csv', 'barometer_new.csv', 'Wildhorn_Telemetry_CLEAN.csv', 'Wildhorn_Telemetry_RAW.csv']


In [5]:
paths_to_combine = ["./accelerometer_new.csv",
                     "./barometer_new.csv",
                    "./gnss_data_new.csv"]
print (f"Paths to combine: {len(paths_to_combine)}")


Paths to combine: 3


In [16]:
theBigones = []
def seperateThedataframes(paths_to_combine):
    for i in paths_to_combine:
        theBigones.append(pd.read_csv(i))
    return theBigones

seperateThedataframes(paths_to_combine)
for i in theBigones:
    print(i.columns)

Index(['Date', 'Time(ms)', 'acceleration'], dtype='object')
Index(['Date', 'Time(ms)', 'pressure(Pa)'], dtype='object')
Index(['altitude(m)', 'velocity(m/s)', 'Time(ms)'], dtype='object')


In [26]:
df_combined = theBigones[2].merge(theBigones[0], on='Time(ms)', how='outer').merge(theBigones[1], on='Time(ms)', how='outer')
df_combined = df_combined.drop_duplicates(subset='Time(ms)', keep='first')
df_combined = df_combined.sort_values(by='Time(ms)')
df_combined = df_combined.fillna(method='ffill')
df_combined = df_combined.fillna(method='bfill')

#remove columns date_x and date_y
df_combined = df_combined.drop(columns=['Date_x', 'Date_y'])

print (f"Combined data: {df_combined.columns}")


#save the file 
df_combined.to_csv('combined_data.csv', index=False)
print(f"Combined data saved to combined_data.csv")

/tmp/ipykernel_414038/316931837.py:1: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_combined = theBigones[2].merge(theBigones[0], on='Time(ms)', how='outer').merge(theBigones[1], on='Time(ms)', how='outer')
/tmp/ipykernel_414038/316931837.py:1: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_combined = theBigones[2].merge(theBigones[0], on='Time(ms)', how='outer').merge(theBigones[1], on='Time(ms)', how='outer')
/tmp/ipykernel_414038/316931837.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_combined = df_combined.fillna(method='ffill')


/tmp/ipykernel_414038/316931837.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_combined = df_combined.fillna(method='bfill')


Combined data: Index(['altitude(m)', 'velocity(m/s)', 'Time(ms)', 'acceleration',
       'pressure(Pa)'],
      dtype='object')
Combined data saved to combined_data.csv
